In [2]:
import pandas as pd
import numpy as np
import gdown 
import scipy.sparse as sps

## Загрузка данных

In [3]:
# Загрузка тестовых данных
gdown.download(url="https://drive.google.com/uc?export=download&id=1Ud6jFto6e7FW5y0LxxwX8afpmTrvA0u-",  
               output="test_transacrion_df.csv",
               quiet=False)
# Загрузка датафрейма с тестовыми данными
test_df = pd.read_csv("test_transacrion_df.csv", index_col=0)
test_all_df = test_df.copy()

Downloading...
From: https://drive.google.com/uc?export=download&id=1Ud6jFto6e7FW5y0LxxwX8afpmTrvA0u-
To: c:\code\rec_sys\hse_mlds_recsys_project\ML\test_transacrion_df.csv
100%|██████████| 52.4M/52.4M [00:04<00:00, 12.5MB/s]


In [4]:
# Загрузка датафрейма с рейтингами
gdown.download(url="https://drive.google.com/uc?export=download&id=1epGrpzB8BEC2t5Od3hrL3x07B1VZIm3c",  
               output="ratings.csv",
               quiet=False)
rating_df = pd.read_csv("ratings.csv", index_col=0)

Downloading...
From (original): https://drive.google.com/uc?export=download&id=1epGrpzB8BEC2t5Od3hrL3x07B1VZIm3c
From (redirected): https://drive.google.com/uc?export=download&id=1epGrpzB8BEC2t5Od3hrL3x07B1VZIm3c&confirm=t&uuid=bd6e5346-37c6-425a-b624-2c28dbe7b3f9
To: c:\code\rec_sys\hse_mlds_recsys_project\ML\ratings.csv
100%|██████████| 211M/211M [00:07<00:00, 26.5MB/s] 


In [5]:
# Оставляем только продукты которые встречались больше N раз
N = 15
top_product = np.array(rating_df.groupby(['product_id'], as_index=False)['rating']
                    .count().query(f'rating > {N}')['product_id'])
rating_df = rating_df[rating_df['product_id'].isin(top_product)]

In [6]:
# Кодировщик пользователей
user2id = {k:v for v, k in enumerate(rating_df["user_id"].unique())}
id2user = {k:v for v, k in user2id.items()}

rating_df['user_id'] = rating_df["user_id"].apply(lambda x: user2id[x])
test_all_df['item_id'] = test_all_df["user_id"].apply(lambda x: user2id.get(x, 999999))

In [7]:
# Приведение типов к менее тяжеловесным
rating_df["user_id"] = pd.to_numeric(rating_df["user_id"], downcast="integer")
rating_df["product_id"] = pd.to_numeric(rating_df["product_id"], downcast="integer")
rating_df["rating"] = pd.to_numeric(rating_df["rating"], downcast="integer")

In [8]:
# Кодировка items
item2id = {k:v for v, k in enumerate(sorted(rating_df['product_id'].unique()))}
id2item = {k:v for v, k in item2id.items()}
rating_df['product_id'] = rating_df['product_id'].apply(lambda x: item2id[x])
test_all_df['product_id'] = test_all_df['product_id'].apply(lambda x: item2id.get(x, -1))

In [9]:
# Удаление повторяющихся покупок в тесте
test_all_df = test_all_df.drop_duplicates(subset=['user_id', 'product_id'], keep='first')

In [10]:
def make_sparse(dataset: pd.DataFrame) -> sps.coo_matrix:
    row = dataset["user_id"].to_numpy()
    col = dataset["product_id"].to_numpy()
    data = dataset["rating"].to_numpy()
    return sps.coo_matrix((data, (row, col)))

## Метрика качества

In [11]:
def precision_at_k(relevant, predicted, k: int = 10):
    """ 
        Функиця расчета Precision@k
        relevant - релевантные items для одного пользователя
        predicted - рекомендованные items для одного пользователя
    """
    return len(set(relevant[:k]) & set(predicted[:k]))/k 

def rel_item(relevant, predicted):
    """
        Функция рассчитывает количество релевантных Item
    """
    result = [0]*max(len(relevant), len(predicted))
    items = min(len(relevant), len(predicted))
    for i in range(items):
        result[i] = int(relevant[i] == predicted[i])
    return result  


def ap_at_k(relevant, predicted, k: int = 10):
    """ 
        Функция расчета AP@k
        relevant - релевантные items для одного пользователя
        predicted - рекомендованные items для одного пользователя
    """
    y_i = rel_item(relevant=relevant, predicted=predicted)
    p_at_i = [0]*k
    iter_cnt = min(len(relevant), k)
    for i in range(1, iter_cnt+1):
        p_at_i[i-1] = precision_at_k(relevant=relevant, predicted=predicted, k=i)
    return sum([y*p/k for y, p in zip(y_i, p_at_i)])

def map_at_k(relevant, predicted, k: int = 10):
    """ 
        Функция расчета MAP@k
        relevant список по всем пользователям с их релевантными items
        predicted список по всем пользователям с их рекомендованными items
    """
    users = len(relevant)
    sum_apk = 0
    for user in range(users):
        sum_apk += ap_at_k(relevant=relevant[user], predicted=predicted[user], k=k)
    return sum_apk/users


def nap_at_k(relevant, predicted, k: int = 10):
    """ 
        Функция расчета normalize AP@k
        relevant - релевантные items для одного пользователя
        predicted - рекомендованные items для одного пользователя
    """
    y_i = rel_item(relevant=relevant, predicted=predicted)
    p_at_i = [0]*k
    k = min(len(relevant), k)
    for i in range(1, k+1):
        p_at_i[i-1] = precision_at_k(relevant=relevant, predicted=predicted, k=i)
    return sum([y*p/k for y, p in zip(y_i, p_at_i)])

def mnap_at_k(relevant, predicted, k: int = 10):
    """ 
        Функция расчета MAP@k
        relevant список по всем пользователям с их релевантными items
        predicted список по всем пользователям с их рекомендованными items
    """
    users = len(relevant)
    sum_napk = 0
    for user in range(users):
        sum_napk += nap_at_k(relevant=relevant[user], predicted=predicted[user], k=k)
    return sum_napk/users


def hitrate_at_k(relevant, predicted, k: int = 10):
    """
        Функция расчета Hitrate@k
        relevant список по всем пользователям с их релевантными items
        predicted список по всем пользователям с их рекомендованными items
    """
    
    cnt_user = len(predicted) # Количество пользователей    
    cnt_valid_user = 0
    for user in range(cnt_user):
        cnt_valid_user += int(len(set(relevant[user][:k]) & set(predicted[user][:k])) > 0) 

    return cnt_valid_user/cnt_user


def ndsg_at_k(relevant, predicted, k: int = 10):
    """
        Функция расчета nDSG@k
        relevant - релевантные items для одного пользователя
        predicted - рекомендованные items для одного пользователя
    """
    idsg_at_k = sum([1/np.log2(k+1) for k in range(1, k+1)])
    k = min(len(relevant), k)
    dsg_at_k = 0
    for k in range(1, k+1):
        dsg_at_k += int(relevant[k-1] == predicted[k-1])/np.log2(k+1)
    return dsg_at_k/idsg_at_k

## Модель

In [12]:
matrix = sps.coo_matrix(
    (np.ones(rating_df.shape[0]), (rating_df['user_id'], rating_df['product_id'])),
    shape=(rating_df["user_id"].nunique(), rating_df["product_id"].nunique()),
)
matrix

<99999x29454 sparse matrix of type '<class 'numpy.float64'>'
	with 9000952 stored elements in COOrdinate format>

In [13]:
class EASE():

    def __init__(self, encoder, rating_df=None, model=None):
        if model is not None:
            self.model = model 
        self.encoder = encoder
        self.rating_df = rating_df

    def predict(self, users_to_recommend: list, k: int = 10, n_jobs: int = 2):
        """
            Параллельное вычисление рекомендаций для пользователей
        """
        predictions = {}
        for uid in users_to_recommend:
            predictions[uid] = self.recommend(uid=uid, k=k)
        return predictions

    def cold_start(self):
        """
            Функция холодного старта
            возвращает популярные продукты по убываюнию
        """
        if self.rating_df is None:
            return None
        return np.argsort(np.array(self.rating_df.groupby("product_id")["rating"].sum()))[::-1]


    def recommend(self, uid: int, k: int):
        """
            Рекомендации 
        """
        uid = self.encoder.get(uid, -1)
        if uid == -1:
            return self.cold_start()[:k]
         
        # Покупки пользователя
        interact = self.rating_df[self.rating_df['user_id'] == uid]['product_id'].to_list()
        # Составляем вектор интеракций человека
        vector = np.zeros(len(item2id))
        vector[interact] = 1
        vector = sps.csr_matrix(vector)
        preds = np.array(vector.dot(self.model))[0]
        ranks = np.argsort(-preds)
        return list(ranks[:k])
         

    def fit(self, X, reg_weight=100):
        """
            Обучаем конечную модель 
        """
        # gram matrix
        G = X.T @ X
        G += reg_weight * sps.identity(G.shape[0]).astype(np.float32)
        # convert to dense because inverse will be dense
        G = G.todense()
        # invert. this takes most of the time
        P = np.linalg.inv(G)
        self.model = P / (-np.diag(P))
        # zero out diag
        np.fill_diagonal(self.model, 0.)


In [14]:
%%time
model = EASE(encoder=user2id, rating_df=rating_df)
model.fit(matrix, reg_weight=10)

CPU times: total: 1h 17min 6s
Wall time: 12min 58s


In [15]:
%%time
# Предсказание по всем пользователям матрицы интеракций
X = matrix.tocsc()
ans = np.array(X.dot(model.model))

CPU times: total: 27min 28s
Wall time: 3h 31min 51s


In [16]:
%%time
predict = []
for i in range(X.shape[0]):
    predict.append(np.argsort(-ans[i])[:10])

CPU times: total: 3min 32s
Wall time: 8min 8s


In [17]:
# Пользователи с их релевантными покупками
relevant = (test_all_df[test_all_df["user_id"].isin(user2id.keys())].sort_values(by=["user_id", "add_to_cart_order"])
            .groupby(['user_id'])
            .agg({'product_id': 'unique'})['product_id'].to_list())

In [18]:
metrics = {}
metrics["MAP@k"] = map_at_k(relevant=relevant, predicted=predict, k=10)
metrics["MNAP@k"] = mnap_at_k(relevant=relevant, predicted=predict, k=10)
metrics["Hitrate@k"] = hitrate_at_k(relevant=relevant, predicted=predict, k=10)
sum_ndsg = 0
sum_precision = 0
for user in range(len(relevant)):
    rel = relevant[user]
    pred = predict[user]
    sum_ndsg += ndsg_at_k(relevant=rel, predicted=pred, k=10)
    sum_precision += precision_at_k(relevant=rel, predicted=pred, k=10)
metrics["AVG_nDSG@k"] = sum_ndsg/len(relevant)
metrics["AVG_Precision@k"] = sum_precision/len(relevant)

In [19]:
metrics

{'MAP@k': 0.007326797871153362,
 'MNAP@k': 0.011928211155064022,
 'Hitrate@k': 0.6323363233632336,
 'AVG_nDSG@k': 0.01797114979938221,
 'AVG_Precision@k': 0.10981009810105222}